# Preprocessing and storing DF in pickle
Perhaps we might Dwh Pat Dn Date later on. 

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import os 
from datetime import timedelta
pd.set_option('display.max_columns', None)

data_path = str(Path(os.getcwd())) + "/data/"

def write_to_pickle(dataframe, name):
    dataframe.to_pickle(data_path + name + ".pickle")
def read_from_pickle(name): 
    return pd.read_pickle(data_path + name + ".pickle")

In [2]:
bio = pd.read_excel('data/20170713_Marc Gliomes_Bio.xlsx', index_col = None)
clin = pd.read_excel('data/20170713_Marc Gliomes_Clin.xlsx', index_col = None)

In [3]:
b = bio.loc[bio['Pat Nip']==4406078178,'Dwh Prv Date Chir']
c = clin.loc[clin['Pat Nip']==4406078178,'Clin Chir Date']

#c.iloc[:len(c)]=1
#clin.loc[clin['Pat Nip']==4406078178,'Clin Chir Date'] = c
#clin.loc[clin['Pat Nip']==4406078178,'Clin Chir Date'] 
#print(clin.loc[clin['Pat Nip']==4406078178,'Clin Chir Date'].iloc[i])
#clin.loc[clin['Pat Nip']==4406078178,'Clin Chir Date']    
#bio['Dwh Prv Date Chir'] = bio['Dwh Prv Date Chir'].apply(lambda x: x.to_pydatetime())
#bio.loc[2,'Dwh Prv Date Chir'].to_pydatetime()-timedelta(days=1)
#clin['Clin Chir Date'] = clin['Clin Chir Date'].apply(lambda x: x.to_pydatetime() + timedelta(days=1))
#bio['Dwh Prv Date Chir'] = bio['Dwh Prv Date Chir'].apply(lambda x: x.to_pydatetime())

Fix the surgery and clinical date so that they match

In [4]:
# Make the two surgery dates match 
# You are taking copies of the dataframe with loc and iloc, so you need to pay attention when you do assignment
# Can definitely remove one or both 'for' loops
for val in np.unique(clin['Pat Nip']): 
    bio_data = bio.loc[bio['Pat Nip']==val,'Dwh Prv Date Chir']
    clin_data = clin.loc[clin['Pat Nip']==val,'Clin Chir Date']
    lc = len(clin_data)
    lb = len(bio_data)
    if lc <=lb:
        for i in range(0,lc):
            clin_data.iloc[i] = bio_data.iloc[i]        
    else: 
        for i in range(0,lb):
            clin_data.iloc[i] = bio_data.iloc[i]
    clin.loc[clin['Pat Nip']==val,'Clin Chir Date'] = clin_data        

In [5]:
df = pd.merge(bio, clin, left_on = ['Pat Nip','Dwh Prv Date Chir'], right_on = ['Pat Nip','Clin Chir Date'],how='outer')
df.columns

Index(['Dwh Prv Nip', 'Dwh Pat Consentement', 'Pat Nip', 'Pat Sexe_x',
       'Pat Date Naissance_x', 'Dwh Pat Dn Date_x', 'Dwh Pat Date Deces_x',
       'Dwh Prv Date Chir', 'Dwh Prv Diag Histologie', 'Grade', 'Dwh Gen Idh1',
       'Dwh Gen Idh2', 'Dwh Gen Htert C228T', 'Dwh Gen Htert C250T',
       'Dwh Gen P53', 'Dwh Gen Mgmt', 'Dwh Gen Egfr', 'Dwh Gen Mdm2',
       'Dwh Gen Cdk4', 'Dwh Gen P16', 'Dwh Ihc Atrx', 'Dwh Chr Ch1P',
       'Dwh Chr Ch19Q', 'Dwh Chr Ch10P', 'Dwh Chr Ch10Q', 'Dwh Chr Ch9P',
       'Dwh Chr Ch9Q', 'Dwh Pat Tumeur Siege', 'Dwh Pat Tumeur Position',
       'Pat Sexe_y', 'Pat Date Naissance_y', 'Dwh Pat Dn Date_y',
       'Dwh Pat Date Deces_y', 'Clin Diag Histologie', 'Grade histo',
       'Clin Chir Date', 'Clin Evolution', 'Clin Type Chirurgie',
       'Clin Rx Date', 'Dwh Chm Date Debut'],
      dtype='object')

Before merging sexes, birthdays and deaths

In [6]:
# Can check before merging the whole thing as well. 
#bio['Pat Sexe'].isnull().value_counts()
print(df['Pat Sexe_x'].isnull().value_counts())
print(df['Pat Date Naissance_x'].isnull().value_counts())
print(df['Dwh Pat Date Deces_x'].isnull().value_counts())

False    7456
True      369
Name: Pat Sexe_x, dtype: int64
False    7725
True      100
Name: Pat Date Naissance_x, dtype: int64
True     4921
False    2904
Name: Dwh Pat Date Deces_x, dtype: int64


# Copy all _y columns into _x: sex, birthday and death, clinical histology, grade histology

In [7]:
for val in np.unique(df['Pat Nip']): 
    x_data = df.loc[df['Pat Nip']==val,'Pat Sexe_x']
    y_data = df.loc[df['Pat Nip']==val,'Pat Sexe_y']
    lc = len(x_data)
    for i in range(0,lc):
        check = x_data.iloc[i]
        if pd.isnull(check):
             x_data.iloc[i] = y_data.iloc[i]        
    df.loc[df['Pat Nip']==val,'Pat Sexe_x'] = x_data

for val in np.unique(df['Pat Nip']): 
    x_data = df.loc[df['Pat Nip']==val,'Pat Date Naissance_x']
    y_data = df.loc[df['Pat Nip']==val,'Pat Date Naissance_y']
    lc = len(x_data)
    for i in range(0,lc):
        check = x_data.iloc[i]
        if pd.isnull(check):
             x_data.iloc[i] = y_data.iloc[i]        
    df.loc[df['Pat Nip']==val,'Pat Date Naissance_x'] = x_data    

for val in np.unique(df['Pat Nip']): 
    x_data = df.loc[df['Pat Nip']==val,'Dwh Pat Date Deces_x']
    y_data = df.loc[df['Pat Nip']==val,'Dwh Pat Date Deces_y']
    lc = len(x_data)
    for i in range(0,lc):
        check = x_data.iloc[i]
        if pd.isnull(check):
             x_data.iloc[i] = y_data.iloc[i]        
    df.loc[df['Pat Nip']==val,'Dwh Pat Date Deces_x'] = x_data   
    
for val in np.unique(df['Pat Nip']): 
    x_data = df.loc[df['Pat Nip']==val,'Dwh Prv Diag Histologie']
    y_data = df.loc[df['Pat Nip']==val,'Clin Diag Histologie']
    lc = len(x_data)
    for i in range(0,lc):
        check = x_data.iloc[i]
        if pd.isnull(check):
             x_data.iloc[i] = y_data.iloc[i]        
    df.loc[df['Pat Nip']==val,'Dwh Prv Diag Histologie'] = x_data   

for val in np.unique(df['Pat Nip']): 
    x_data = df.loc[df['Pat Nip']==val,'Grade']
    y_data = df.loc[df['Pat Nip']==val,'Grade histo']
    lc = len(x_data)
    for i in range(0,lc):
        check = x_data.iloc[i]
        if pd.isnull(check):
             x_data.iloc[i] = y_data.iloc[i]        
    df.loc[df['Pat Nip']==val,'Grade'] = x_data   

for val in np.unique(df['Pat Nip']): 
    x_data = df.loc[df['Pat Nip']==val,'Dwh Pat Dn Date_x']
    y_data = df.loc[df['Pat Nip']==val,'Dwh Pat Dn Date_y']
    lc = len(x_data)
    for i in range(0,lc):
        check = x_data.iloc[i]
        if pd.isnull(check):
             x_data.iloc[i] = y_data.iloc[i]        
    df.loc[df['Pat Nip']==val,'Dwh Pat Dn Date_x'] = x_data   


In [9]:
# A small improvement
print(df['Pat Sexe_x'].isnull().value_counts())
print(df['Pat Date Naissance_x'].isnull().value_counts())
print(df['Dwh Pat Date Deces_x'].isnull().value_counts())
print(df['Dwh Prv Diag Histologie'].isnull().value_counts())
# Testing for empty values with an example to make sure merge is correct 
#clin.loc[clin['Pat Nip']==1111112776, 'Pat Sexe'].values
#clin['Pat Sexe'].value_counts()
#clin.loc[clin['Pat Sexe'].isnull()==True,'Pat Nip']

False    7493
True      332
Name: Pat Sexe_x, dtype: int64
False    7762
True       63
Name: Pat Date Naissance_x, dtype: int64
True     4908
False    2917
Name: Dwh Pat Date Deces_x, dtype: int64
False    7825
Name: Dwh Prv Diag Histologie, dtype: int64


In [10]:
df.rename(columns = {'Pat Nip':'ID','Pat Sexe_x':'Sex','Dwh Pat Dn Date_x':'Diagnostic_date','Pat Date Naissance_x':'DoB','Dwh Pat Date Deces_x':'Death_date',
                      'Dwh Prv Date Chir':'Surgery_date', 'Dwh Prv Diag Histologie':'Tumor_type','Grade':'Tumor_grade',
                      'Dwh Gen Idh1':'Gene_Idh1', 'Dwh Gen Idh2':'Gene_Idh2','Dwh Gen Htert C228T':'Gene_Htert_C228T', 
                      'Dwh Gen Htert C250T':'Gene_Htert_C250T', "Dwh Gen P53":'Gene_P53', 'Dwh Gen Mgmt':'Gene_Mgmt', 
                      'Dwh Gen Egfr':'Gene_Egfr', 'Dwh Gen Mdm2':'Gene_Mdm2','Dwh Gen Cdk4':'Gene_Cdk4',
                      'Dwh Gen P16':'Gene_P16', 'Dwh Ihc Atrx':'Gene_Ihc_Atrx', 'Dwh Chr Ch1P':'Gene_Ch1P',
                      'Dwh Chr Ch19Q':'Gene_Ch19Q', 'Dwh Chr Ch10P':'Gene_Ch10P', 'Dwh Chr Ch10Q':'Gene_Ch10Q', 'Dwh Chr Ch9P':'Gene_Ch9P',
                      'Dwh Chr Ch9Q':'Gene_Ch9Q', 'Dwh Pat Tumeur Siege':'Tumor_Location', 'Dwh Pat Tumeur Position':'Tumor_Position',
                      'Clin Evolution':'Clinical_evolution', 'Clin Type Chirurgie':'Surgery_type',
                      'Clin Rx Date':'Rx_date', 'Dwh Chm Date Debut':'Chemo_start_date'}, inplace = True)
df.drop(['Dwh Pat Dn Date_y','Pat Sexe_y','Dwh Pat Date Deces_y','Pat Date Naissance_y'], axis=1,inplace=True)
df.drop('Dwh Pat Consentement', axis=1, inplace=True)
df.drop('Dwh Prv Nip', axis=1, inplace=True)
df.drop(['Clin Diag Histologie', 'Grade histo','Clin Chir Date',],axis=1,inplace=True)

# Diagnostic very sparse, 2000 entries out of 7000
#interesting to analyze surgery date, all other things being equal. or any other proxy of impact of surgeon?
#what is the right measure of survival? date of diagnosis? date of surgery? none marks the actual 'beginning'. survival
#post diagnosis after controlling for tumor stage?

In [11]:
df.rename(columns = {'Sex':'Gender'},inplace = True) 
write_to_pickle(df,'post_merge_data')


,ID,Gender,DoB,Diagnostic_date,Death_date,Surgery_date,Tumor_type,Tumor_grade,Gene_Idh1,Gene_Idh2,Gene_Htert_C228T,Gene_Htert_C250T,Gene_P53,Gene_Mgmt,Gene_Egfr,Gene_Mdm2,Gene_Cdk4,Gene_P16,Gene_Ihc_Atrx,Gene_Ch1P,Gene_Ch19Q,Gene_Ch10P,Gene_Ch10Q,Gene_Ch9P,Gene_Ch9Q,Tumor_Location,Tumor_Position,Clinical_evolution,Surgery_type,Rx_date,Chemo_start_date
0,4,M,1940-05-22,NaN,2001-02-22,2000-05-30,glioblastome IV,4.0,NC,NC,NaN,NaN,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,autre,G,P1,exérèse,2000-07-07,NaT
1,5,M,1947-10-15,NaN,1999-12-15,1999-05-29,glioblastome IV,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sus-T hémisp,G,TT ini,biopsie,1999-07-12,1999-06-29


In [44]:
df = read_from_pickle('post_merge_data')

In [26]:
df.head(2)

,ID,Gender,DoB,Diagnostic_date,Death_date,Surgery_date,Tumor_type,Tumor_grade,Gene_Idh1,Gene_Idh2,Gene_Htert_C228T,Gene_Htert_C250T,Gene_P53,Gene_Mgmt,Gene_Egfr,Gene_Mdm2,Gene_Cdk4,Gene_P16,Gene_Ihc_Atrx,Gene_Ch1P,Gene_Ch19Q,Gene_Ch10P,Gene_Ch10Q,Gene_Ch9P,Gene_Ch9Q,Tumor_Location,Tumor_Position,Clinical_evolution,Surgery_type,Rx_date,Chemo_start_date
0,4,M,1940-05-22,NaN,2001-02-22,2000-05-30,glioblastome IV,4.0,NC,NC,NaN,NaN,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,autre,G,P1,exérèse,2000-07-07,NaT
1,5,M,1947-10-15,NaN,1999-12-15,1999-05-29,glioblastome IV,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sus-T hémisp,G,TT ini,biopsie,1999-07-12,1999-06-29


# End of merging. Next we deal with nan/empty values

In [27]:
#Multiple entries for the same person, can we treat them independently?
#df.loc[df.ID==4204007149]

In [45]:
df_clean = df.copy(deep=True)

In [46]:
df_clean = df.copy(deep=True)

# Retain observations for which either Idh1 or Idh2 are null but not both - why not remove nulls altogether?
df_clean = df_clean.loc[(df_clean.Gene_Idh2.notnull()) | (df_clean.Gene_Idh1.notnull())]
#df_clean = df_clean[pd.notnull(df.Gene_Idh2)]
#df_clean = df_clean[pd.notnull(df.Gene_Idh1)]

df_clean= df_clean[df_clean.DoB.notnull()]
df_clean = df_clean[df_clean.Gender.notnull()]

# Remove diagnostic dates as Pr. Marc said they are useless
df_clean.drop('Diagnostic_date', axis=1, inplace=True)

In [47]:
# Sanity check
(df_clean.Gene_Idh1.isnull() & df_clean.Gene_Idh2.isnull()).value_counts()

False    7057
dtype: int64

In [48]:
# Remove NaN for surgery dates before creating new variable
df_clean = df_clean[df_clean.Surgery_date.notnull()]
df_clean.Surgery_date.isnull().value_counts()

False    6794
Name: Surgery_date, dtype: int64

In [49]:
# Add age at surgery variable
df_clean['Age_surgery'] = df_clean['Surgery_date'] - df_clean['DoB']

# Time manipulations

In [50]:
df_clean.Death_date.isnull().value_counts()

True     3995
False    2799
Name: Death_date, dtype: int64

In [51]:
# Convert Age at surgery to years
df_clean['Age_surgery'] = df_clean['Age_surgery'] / np.timedelta64(1, 'Y')
df_clean['Age_surgery'] = df_clean['Age_surgery'].round()

Two approaches: 
- we remove null values for death dates
- we deal with null values for life_expectancy later on

In [52]:
# Comment the first line to have the second approach
df_clean = df_clean[df_clean.Death_date.notnull()]
df_clean['life_expectancy'] = df_clean.Death_date - df_clean.Surgery_date
df_clean['life_expectancy'] = df_clean['life_expectancy'] / np.timedelta64(1, 'D')

In [53]:
df_clean_test = df_clean.copy(deep=True)

In [54]:
#implementing clinical interaction variable between IDH and TERT
df_clean_test.loc[((df_clean_test['Gene_Idh1'] == 'ALTERE') | (df_clean_test['Gene_Idh2'] == 'ALTERE')) & ((df_clean_test['Gene_Htert_C228T'] == "ALTERE") | (df_clean_test['Gene_Htert_C250T'] == "ALTERE")), "IDH_TERT"] = 1
df_clean_test.loc[((df_clean_test['Gene_Idh1'] == 'ALTERE') | (df_clean_test['Gene_Idh2'] == 'ALTERE')) & ((df_clean_test['Gene_Htert_C228T'] == "NORMAL") | (df_clean_test['Gene_Htert_C250T'] == "NORMAL")), "IDH_TERT"] = 2
df_clean_test.loc[((df_clean_test['Gene_Idh1'] == 'NORMAL') | (df_clean_test['Gene_Idh2'] == 'NORMAL')) & ((df_clean_test['Gene_Htert_C228T'] == "NORMAL") | (df_clean_test['Gene_Htert_C250T'] == "NORMAL")), "IDH_TERT"] = 3
df_clean_test.loc[((df_clean_test['Gene_Idh1'] == 'NORMAL') | (df_clean_test['Gene_Idh2'] == 'NORMAL')) & ((df_clean_test['Gene_Htert_C228T'] == "ALTERE") | (df_clean_test['Gene_Htert_C250T'] == "ALTERE")), "IDH_TERT"] = 4

In [55]:
# We keep only the first record of patients with multiple surgeries
df_clean_test['ID'].duplicated().value_counts()

False    2204
True      595
Name: ID, dtype: int64

In [56]:
# Sort by ID and Surgery date, in order to keep only earliest surgery date
df_clean_test.sort_values(['ID', 'Surgery_date'], inplace=True)

In [57]:
# Indicate whether duplicate and check sum of them
df_clean_test['multiple_surgeries'] = df_clean_test.duplicated(['ID'])
df_clean_test['multiple_surgeries'].sum()

595

In [69]:
# Reduce to observations that don't have multiple surgeries

df_clean_test.drop_duplicates('ID',keep='first',inplace=True)
df_clean_test.head(5)

,ID,Gender,DoB,Death_date,Surgery_date,Tumor_type,Tumor_grade,Gene_Idh1,Gene_Idh2,Gene_Htert_C228T,Gene_Htert_C250T,Gene_P53,Gene_Mgmt,Gene_Egfr,Gene_Mdm2,Gene_Cdk4,Gene_P16,Gene_Ihc_Atrx,Gene_Ch1P,Gene_Ch19Q,Gene_Ch10P,Gene_Ch10Q,Gene_Ch9P,Gene_Ch9Q,Tumor_Location,Tumor_Position,Clinical_evolution,Surgery_type,Rx_date,Chemo_start_date,Age_surgery,life_expectancy,IDH_TERT,multiple_surgeries
0,4,M,1940-05-22,2001-02-22,2000-05-30,glioblastome IV,4.0,NC,NC,NaN,NaN,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,autre,G,P1,exérèse,2000-07-07,NaT,60.0,268.0,NaN,False
2,10,M,1949-11-05,1992-05-15,1992-04-15,gliome mixte ana III,3.0,ALTERE,NORMAL,NORMAL,NORMAL,NC,NaN,NORMAL,NaN,NaN,NaN,NaN,NON PERTE,NON PERTE,NON PERTE,PERDU,NON PERTE,NaN,sus-T hémisp,G,TT ini,partielle,1992-05-01,1992-05-01,42.0,30.0,3.0,False
3,11,M,1958-07-29,1991-11-15,1986-03-17,astrocytome bas grade II,2.0,NC,NC,NaN,NaN,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,non précisé,non précisé,TT ini,exérèse,1986-04-01,NaT,28.0,2069.0,NaN,False
5,12,F,1930-12-07,1991-03-13,1990-02-13,glioblastome IV,4.0,NORMAL,NaN,NORMAL,ALTERE,NORMAL,NON METHYLE,NORMAL,NORMAL,NORMAL,DELETE,NaN,PARTIEL,PARTIEL,PERDU,PERDU,PERDU,NaN,sus-T hémisp,G,TT ini,aucune,1990-03-14,NaT,59.0,393.0,4.0,False
7,13,M,1928-11-17,1990-12-20,1989-11-17,glioblastome IV,4.0,NORMAL,NaN,NORMAL,NORMAL,NORMAL,NON METHYLE,NORMAL,NORMAL,NORMAL,NORMAL,NaN,NON PERTE,NON PERTE,NON PERTE,PERDU,NaN,NaN,sus-T hémisp,D,TT ini,partielle,1989-12-18,NaT,61.0,398.0,3.0,False


In [81]:
df_final = df_clean_test.copy(deep=True)
df_final.multiple_surgeries.isnull().value_counts()

False    2204
Name: multiple_surgeries, dtype: int64

In [82]:
# They seem to have been datetime objects without the below conversions?
# See here why we need to use loc: 
# https://stackoverflow.com/questions/26724378/pandas-settingwithcopywarning?noredirect=1&lq=1

df_final.loc[:,'Rx_date'] = pd.to_datetime(df_final.Rx_date,errors = 'coerce')
df_final.loc[:,'Chemo_start_date'] = pd.to_datetime(df_final.Chemo_start_date,errors = 'coerce')

df_final.loc[:,'Rx_date'] = df_final['Rx_date'].dt.year
df_final.loc[:,'Chemo_start_date']=df_final.Chemo_start_date.dt.year

In [83]:
# Useful
#for col in df_clean.columns:
 #    print('{0}\n  {1}\n'.format(col,df_clean[col].isnull().value_counts()))

# Merge KPS dataset

In [84]:
# Import KPS dataset to merge it
kps = pd.read_excel('data/KPS.xlsx')

In [86]:
# Sort by ID and surgery date, as above
kps.sort_values(['NIP', 'CHIR_DATE'],inplace=True)

In [87]:
# Reduce to single observations as with multiple surgeries from previous dataset
kps.drop_duplicates('NIP',keep='first',inplace=True)

In [89]:
kps.shape

(6772, 6)

In [93]:
# Keep only columns that are of interest or needed for the merge
kps_sorted_final = kps[['NIP', 'ID_EC', 'IK_CLINIQUE', 'CHIR_DATE', 'CHIR_IK_PRE', 'RX_IK_PRE']]

In [96]:
# Merge the two datasets using surgery date and chirurgie date, only the later is not a datetime.
# Use ID for merge as well 
kps.loc[:,'CHIR_DATE'] = pd.to_datetime(kps.CHIR_DATE, errors = 'coerce')

In [97]:
df_final_kps = pd.merge(df_final, kps, how = 'left', left_on = ['ID', 'Surgery_date'], right_on = ['NIP', 'CHIR_DATE'])

In [98]:
len(df_final)

2204

In [99]:
len(df_final_kps)

2204

# Looks good so far 

In [100]:
# Code Rx date into either 1 if date is present, or leave it as missing otherwise
df_final_kps['RX'] = np.where(df_final_kps['Rx_date'].notnull(), 1, np.NaN)

In [101]:
df_final_kps['CHEM'] = np.where(df_final_kps['Chemo_start_date'].notnull(), 1, np.NaN)

In [65]:
#drop NIP as it's duplicate of ID after the merge
#drop multiple surgeries as it was only needed to remove duplicates
#drop CHIR Date as it comes from KPS dataset after merge and is duplicate of Surgery_date
#drop Rx_date and Chemo_start_date
#Final dataset before imputation
#df_final_kps.drop('NIP', axis = 1, inplace=True)
#df_final_kps.drop('multiple_surgeries', axis = 1, inplace=True)
#df_final_kps.drop('CHIR_DATE', axis = 1, inplace=True)
#df_final_kps.drop('Chemo_start_date', axis = 1, inplace=True)
#df_final_kps.drop('Rx_date', axis = 1, inplace=True)

# Keep only observations for which either the death date is observed, or which had RX or CHEMO in 2017
# I find it easier to use dataframe_name[pd.notnull(df.x)]
#df_final_kps = df_final_kps.loc[(df_final_kps.Death_date.notnull())]

In [102]:
df_final_kps.head(2)

,ID,Gender,DoB,Death_date,Surgery_date,Tumor_type,Tumor_grade,Gene_Idh1,Gene_Idh2,Gene_Htert_C228T,Gene_Htert_C250T,Gene_P53,Gene_Mgmt,Gene_Egfr,Gene_Mdm2,Gene_Cdk4,Gene_P16,Gene_Ihc_Atrx,Gene_Ch1P,Gene_Ch19Q,Gene_Ch10P,Gene_Ch10Q,Gene_Ch9P,Gene_Ch9Q,Tumor_Location,Tumor_Position,Clinical_evolution,Surgery_type,Rx_date,Chemo_start_date,Age_surgery,life_expectancy,IDH_TERT,multiple_surgeries,NIP,ID_EC,CHIR_DATE,IK_CLINIQUE,CHIR_IK_PRE,RX_IK_PRE,RX,CHEM
0,4,M,1940-05-22,2001-02-22,2000-05-30,glioblastome IV,4.0,NC,NC,NaN,NaN,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,autre,G,P1,exérèse,2000.0,NaN,60.0,268.0,NaN,False,NaN,NaN,NaT,NaN,NaN,NaN,1.0,NaN
1,10,M,1949-11-05,1992-05-15,1992-04-15,gliome mixte ana III,3.0,ALTERE,NORMAL,NORMAL,NORMAL,NC,NaN,NORMAL,NaN,NaN,NaN,NaN,NON PERTE,NON PERTE,NON PERTE,PERDU,NON PERTE,NaN,sus-T hémisp,G,TT ini,partielle,1992.0,1992.0,42.0,30.0,3.0,False,10.0,TTini,1992-04-15,NaN,NaN,NaN,1.0,1.0


In [103]:
#df_final_kps['life_expectancy'] = pd.to_datetime(df_final_kps['life_expectancy'])
df_final_kps['life_expectancy'] = df_final_kps['life_expectancy'].round()

# Create Imputation dataset

In [106]:
df_final_kps.drop(['DoB','Death_date','Surgery_date','Chemo_start_date','Rx_date'],axis=1,inplace=True)
write_to_pickle(df_final_kps,"preimputation")

In [107]:
# Drop ID so that MICE imputation works
df_final_kps.drop(['ID'], axis=1,inplace=True)

In [108]:
imputation_set = df_final_kps

In [109]:
imputation_set.to_csv('data/imputation_set.csv')
    

In [110]:
#checking how many observations we have with different numbers of NA data across variables
no_NAN = df_final_kps.dropna(thresh = 35)
len(no_NAN)

6

In [111]:
len(df_final_kps.columns)

36

# Add it to a csv ready for imputation. Then read the imputed one for one last manipulation

In [85]:
df_final_kps.to_csv('data/imputation_dataset_22012018.csv')

In [531]:
imputed_dataset.drop(['Unnamed: 0', 'X'], axis = 1, inplace=True)

In [532]:
imputed_dataset

,Gender,Tumor_type,Tumor_grade,Gene_Idh1,Gene_Idh2,Gene_Htert_C228T,Gene_Htert_C250T,Gene_P53,Gene_Mgmt,Gene_Egfr,Gene_Mdm2,Gene_Cdk4,Gene_P16,Gene_Ihc_Atrx,Gene_Ch1P,Gene_Ch19Q,Gene_Ch10P,Gene_Ch10Q,Gene_Ch9P,Gene_Ch9Q,Tumor_Location,Tumor_Position,Clinical_evolution,Surgery_type,Age_surgery,life_expectancy,IDH_TERT,ID_EC,IK_CLINIQUE,CHIR_IK_PRE,RX_IK_PRE
0,M,glioblastome IV,4,NC,NC,NORMAL,ALTERE,NC,METHYLE,AMPLIFIE,AMPLIFIE,NORMAL,DELETE,Maintien,PERDU,PERDU,GAIN,GAIN,GAIN,PERDU,autre,G,P1,exérèse,60,268,4,P1,60,80,80
1,M,gliome mixte ana III,3,ALTERE,NORMAL,NORMAL,NORMAL,NC,METHYLE,NORMAL,NORMAL,NORMAL,NORMAL,Extinction diffuse,NON PERTE,NON PERTE,NON PERTE,PERDU,NON PERTE,NON PERTE,sus-T hémisp,G,TT ini,partielle,42,30,3,TTini,70,80,90
2,M,astrocytome bas grade II,2,NC,NC,NC,NC,NC,METHYLE,AMPLIFIE,AMPLIFIE,AMPLIFIE,DELETE,Extinction diffuse,PERDU,PERDU,GAIN,GAIN,GAIN,GAIN,non précisé,non précisé,TT ini,exérèse,28,2069,4,TTini,75,80,80
3,F,glioblastome IV,4,NORMAL,NORMAL,NORMAL,ALTERE,NORMAL,NON METHYLE,NORMAL,NORMAL,NORMAL,DELETE,Maintien,PARTIEL,PARTIEL,PERDU,PERDU,PERDU,PERDU,sus-T hémisp,G,TT ini,aucune,59,393,4,TTini,60,70,70
4,M,glioblastome IV,4,NORMAL,NORMAL,NORMAL,NORMAL,NORMAL,NON METHYLE,NORMAL,NORMAL,NORMAL,NORMAL,Maintien,NON PERTE,NON PERTE,NON PERTE,PERDU,PERDU,NON PERTE,sus-T hémisp,D,TT ini,partielle,61,398,3,TTini,60,60,80
5,F,glioblastome avec oligo. IV,4,NORMAL,NORMAL,NORMAL,ALTERE,NORMAL,METHYLE,AMPLIFIE,NORMAL,NORMAL,NORMAL,Maintien,GAIN,NON PERTE,NON PERTE,PERDU,NON PERTE,NON PERTE,sus-T hémisp,D,TT ini,exérèse,58,433,4,TTini,60,70,70
6,M,glioblastome IV,4,NORMAL,NORMAL,ALTERE,NORMAL,ALTERE,NON METHYLE,AMPLIFIE,NORMAL,NORMAL,DELETE,Maintien,PARTIEL,NON PERTE,NON PERTE,PERDU,NON PERTE,NON PERTE,sus-T hémisp,G,TT ini,exérèse,56,143,4,TTini,60,80,80
7,M,glioblastome IV,4,NORMAL,NORMAL,NORMAL,NORMAL,NORMAL,METHYLE,NORMAL,NORMAL,NORMAL,DELETE,Maintien,PARTIEL,PARTIEL,PARTIEL,PERDU,NON PERTE,NON PERTE,non précisé,G,TT ini,exérèse,69,408,3,TTini,60,100,70
8,M,glioblastome IV,4,NORMAL,NORMAL,NORMAL,ALTERE,NC,METHYLE,NORMAL,NORMAL,NORMAL,NORMAL,Extinction diffuse,NON PERTE,NON PERTE,NON PERTE,PERDU,NON PERTE,NON PERTE,non précisé,non précisé,TT ini,exérèse,66,7,4,TTini,70,90,70
9,F,astrocytome anaplasique III,3,NORMAL,NORMAL,ALTERE,NORMAL,NORMAL,NON METHYLE,NORMAL,NORMAL,NORMAL,NORMAL,Maintien,NON PERTE,NON PERTE,NON PERTE,NON PERTE,NON PERTE,NON PERTE,sus-T hémisp,G,TT ini,partielle,61,316,4,TTini,80,70,80


In [546]:
imputed_dataset.shape

(2218, 31)

In [543]:
final_dataset_analysis.shape

(4436, 32)

In [545]:
df_final_kps.shape

(2218, 42)

In [547]:
imputed_dataset['ID'] = df_final_kps['ID']

In [548]:
imputed_dataset.shape

(2218, 32)

In [549]:
final_dataset_analysis = imputed_dataset

In [550]:
final_dataset_analysis.to_csv("final_dataset_analysis.csv")